In [2]:
using TensorDecomposition, Symbolics, LinearAlgebra, InvertedIndices

In [53]:
function rlim(n, d)
    if iseven(d)
         if n == 2
             return binomial(Int(n+d/2), n) - n
         else
             return binomial(Int(n+d/2), n) - n - 1
         end
     else 
         return binomial(Int(n+(d-1)/2), n)
     end
 end;

function cofactor(A::AbstractMatrix)
    ax = axes(A)
    out = similar(A, eltype(A), ax)
    for col in ax[1]
        for row in ax[2]
            out[col, row] = (-1)^(col + row) * det(A[Not(col), Not(row)])
        end
    end
    return out
end;

function alpha_iterator(::Val{N}, s, t=()) where {N}
    N <= 1 && return ((s, t...),) # Iterator over a single Tuple
    Iterators.flatten(alpha_iterator(Val(N-1), s-i, (i, t...)) for i in 0:s)
end

function monomialOrder(a, b)
    if a[1] > b[1]
        return true
    elseif a[1] < b[1]
        return false
    else 
        if maximum(a[2:end]) < maximum(b[2:end])
            return true
        elseif maximum(a[2:end]) > maximum(b[2:end])
            return false
        else
            return !isless(a[2:end], b[2:end])
        end 
    end
end

function convertIndices(x)
    d = sum(x)
    y = zeros(Int, d)
    c = 1
    for i=1:length(x)
        pow = x[i]
        y[c:c+pow-1] .= i
        c += pow
    end 
    return y .- 1
end;

function e(j, n)
    e = zeros(n)
    e[j] = 1
    return e
end;

function multBasis(basis, j, n)
    ej = e(j, n)
    ej[1] = -1
    return [Tuple(Int(y) for y in [x for x in b] + ej) for b in basis]
end

multMonomial(b, ej) = Tuple(Int(y) for y in [x for x in b] + ej);

### Real tensor

In [80]:
n = 2
d = 6;

In [81]:
r = rlim(n, d)

8

In [82]:
T, A_, L = randomRankedTensor(n+1, d, r, real=true);

In [83]:
Thank = hankMat2(T);

In [84]:
Thank

28×28 Matrix{Num}:
 -162.699     -157.775       12.465     …     16.6387    -73.1322    -17.4582
 -157.775     -150.759       10.9386        h[4]        h[7]        h[6]
   12.465       10.9386      -0.737764      h[6]        h[5]        h[8]
   10.9386       9.55976     -0.541296     h[13]       h[14]       h[15]
 -150.759     -135.588        9.55976      h[11]       h[16]       h[13]
   -0.737764    -0.541296     0.105375  …  h[15]       h[12]       h[17]
    9.55976      6.97664     -0.218718     h[21]       h[24]       h[23]
   -0.541296    -0.218718    -0.289232     h[23]       h[22]       h[25]
 -135.588     -120.566        6.97664      h[19]       h[26]       h[21]
    0.105375    -0.289232     0.792951     h[25]       h[20]       h[27]
   -0.218718     0.420882     0.914804  …  h[32]       h[33]       h[34]
    6.97664      4.58048      0.420882     h[30]       h[35]       h[32]
   -0.289232     0.914804    -1.87855      h[34]       h[31]       h[36]
    ⋮                      

In [85]:
monomials = sort(collect(alpha_iterator(Val(n+1), Int(d))), lt=monomialOrder);
D = Dict()
for (i, mon) in enumerate(monomials)
   D[mon] = i 
end

basis = monomials[1:r];

In [86]:
first_r = [D[b] for b in basis];

In [87]:
H0 = Thank[first_r, first_r]

8×8 Matrix{Num}:
 -162.699     -157.775     12.465     …  -0.737764    9.55976    -0.541296
 -157.775     -150.759     10.9386       -0.541296    6.97664    -0.218718
   12.465       10.9386    -0.737764      0.105375   -0.218718   -0.289232
   10.9386       9.55976   -0.541296     -0.289232    0.420882    0.914804
 -150.759     -135.588      9.55976      -0.218718    4.58048     0.420882
   -0.737764    -0.541296   0.105375  …   0.792951    0.914804   -1.87855
    9.55976      6.97664   -0.218718      0.914804  -10.8775      2.06438
   -0.541296    -0.218718  -0.289232     -1.87855     2.06438   -19.5139

In [88]:
H0_adj = cofactor(Symbolics.value.(H0))

8×8 Matrix{Float64}:
   111.526     44.9034   -7866.7        …    40.8783      11.7899
    44.9034   -36.5917    2239.56           -66.4705     -26.1109
 -7866.7     2239.56         2.22436e5      568.65       464.018
   185.832   -858.448    23697.0           -335.211     -104.514
  -803.188    111.282    26384.2             50.4901      45.5
  1006.84    -548.417   -18985.7        …  -215.9        -45.0657
    40.8783   -66.4705     568.65            20.3173      -0.509531
    11.7899   -26.1109     464.018           -0.509531    20.4454

In [89]:
Hs = []
for i=2:n+1
    col_inds = [D[b] for b in multBasis(basis, i, n+1)]
    push!(Hs, Thank[first_r, col_inds])
end

In [91]:
monomialDegree(alpha) = sum(alpha[2:end]);

In [92]:
degree_d2 = []
degree_d2_1 = []

for b in basis
   if monomialDegree(b) == d/2-1
        push!(degree_d2_1, b)
    elseif monomialDegree(b) == d/2
        push!(degree_d2, b)
    end
end

In [79]:
linearEquations[1]

0.002975254919612924h[1] - 0.0006168190967158899 - 0.0008065174872010639h[2]

In [77]:
eqMat1 = Hs[1]*H0_adj*Hs[2]
eqMat2 = Hs[2]*H0_adj*Hs[1]
eqMat = eqMat1-eqMat2;

In [94]:
Symbolics.expand.(eqMat)

4×4 Matrix{Num}:
 -1.23686e-15                                      …  -3.94075e-15 - 9.50628e-16h[1] - 7.02563e-16h[2]
  7.28584e-17                                          0.00297525h[1] - 0.000616819 - 0.000806517h[2]
 -2.44032e-15                                          0.0107578h[1] - 0.0025244 - 0.00297525h[2]
 -5.19029e-15 - 1.33227e-15h[1] - 5.27356e-16h[2]      5.82867e-15 + 2.0331e-15h[1] + 5.71591e-16h[2]

In [97]:
beta = degree_d2_1[2]

(4, 2, 0)

In [98]:
D[beta]

5